## Pandas funciton API
Spark有一些函数可以让python的函数通过pandas实例直接用在spark dataframe上。内部机制上类似`pandas udf`, jvm把数据转成arrow的buffer, 然后pandas可以直接在buffer上操作。但是区别是，这些函数api使用起来就像普通pyspark api一样是作用在dataframe上的, 而不像udf那样作用于一个`column`. 实际使用的时候，一般写法是`DataFrame.groupby().applyInPandas()`或者`DataFrame.groupby().mapInPandas()`

### Grouped map api
Spark的dataframe在`groupby`后使用普通的pandas函数， 如`df.groupby().applyInPandas(func, schema))`， 普通的pandas函数需要输入是pandas dataframe, 返回普通的pandas dataframe. 上面这写法会把每个分组group映射到pandas dataframe.
`df.groupby().applyInPandas(func, schema))`过程其实分为三步， 典型的`split-apply-combine`模式：
- `DataFrame.groupBy`分组数据
- 分组的数据映射到pandas dataframe后，apply到传入的函数
- 组合结果成一个新的pyspark Dataframe

使用groupBy().applyInPandas(), 用户需要做两件事：
- 写好pandas函数,输入dataframe, 输出dataframe
- 定义好pyspark dataframe结果的schema


In [2]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("app1") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

df = spark.createDataFrame([(1,1.0), (1,2.0),(2,3.0),(2,4.0),(2,10.0)], ('id','v'))
def substract_mean(pdf):
    v = pdf.v
    return pdf.assign(v=v-v.mean())

df.groupby('id').applyInPandas(substract_mean, schema='id long, v double').show()


+---+------------------+
| id|                 v|
+---+------------------+
|  1|              -0.5|
|  1|               0.5|
|  2|-2.666666666666667|
|  2|-1.666666666666667|
|  2| 4.333333333333333|
+---+------------------+



### map api
也可以对pyspark dataframe和pandas dataframe做map操作，`DataFrame.mapInPandas()`是对当前的DataFrame的取一个迭代器映射普通到pandas函数。这个普通pandas函数必须是输入输出都是pdf.


In [3]:
def filter_func(iterator):
    for pdf in iterator:
        yield pdf[pdf.id==1]
        
df.mapInPandas(filter_func, schema=df.schema).show()


+---+---+
| id|  v|
+---+---+
|  1|1.0|
|  1|2.0|
+---+---+



### co-grouped map api
这个api可以使两个pyspark dataframe组合后使用pandas函数

In [5]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0), (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ("time", "id", "v1"))

df2 = spark.createDataFrame(
    [(20000101, 1, "x"), (20000101, 2, "y")],
    ("time", "id", "v2"))
df1.show()
df2.show()
def asof_join(l, r):
    return pd.merge_asof(l, r, on="time", by="id")

df1.groupby("id").cogroup(df2.groupby("id")).applyInPandas(
    asof_join, schema="time int, id int, v1 double, v2 string").show()

+--------+---+---+
|    time| id| v1|
+--------+---+---+
|20000101|  1|1.0|
|20000101|  2|2.0|
|20000102|  1|3.0|
|20000102|  2|4.0|
+--------+---+---+

+--------+---+---+
|    time| id| v2|
+--------+---+---+
|20000101|  1|  x|
|20000101|  2|  y|
+--------+---+---+

+--------+---+---+---+
|    time| id| v1| v2|
+--------+---+---+---+
|20000101|  1|1.0|  x|
|20000102|  1|3.0|  x|
|20000101|  2|2.0|  y|
|20000102|  2|4.0|  y|
+--------+---+---+---+

